# Implementing the actual replicator dynamics in Franke and Correia 2018

https://github.com/josepedrocorreia/vagueness-games/blob/master/newCode/vagueness-games.py#L291

In [11]:
import torch

In [12]:
def normalize_rows(mat: torch.Tensor):
    """Normalize each row of 2D array / tensor to sum to 1.0."""
    return torch.nan_to_num(mat / mat.sum(1, keepdim=True))

In [13]:
# For simplicity, let's make things like michael's example.
sender_matrix = torch.tensor(
    [
        # [1., 0.,],
        [0.5, 0.5,],
        [1., 0.,],
        [0., 1.,],
        [0., 1.,],
    ]
)
receiver_matrix = torch.tensor(
    [
    # [1., 0., 0., 0.,],
    [0.25, 0.25, 0.25, 0.25],
    [0., 0., 1., 0.,],
    ]
)

utility_matrix = torch.eye(4)
confusion_matrix = torch.eye(4)
prior = torch.tensor([.4, .4, .1, .1,])
num_states = 4
num_signals = 2

In [14]:
P = sender_matrix  # `[states, signals]`
Q = receiver_matrix  # `[signals, states]`
U = utility_matrix  # `[states, states]`
C = confusion_matrix  # `[states, states]`, compare self.game.meaning_dists
p = prior  # `[states,]`

In [15]:
print(P)
O = normalize_rows(p * C) # prob actual state (col) given observed state (row)
Sigma = C @ P # prob signal (col) given actual state (row)
O_sender = O @ Sigma # P_o(w|m_o) prob of seeing agent j send signal given agent i sees state m_o
Rho = Q @ C # prob receiver chooses state (column) given signal (row)
# Expected utility for sender: sum O[m_o, m_a] * Rho[m, m_r] * U[m_a, m_r]
U_sender = torch.einsum('oa,wr,ar->ow', O, Rho, U)
P = normalize_rows(O_sender * U_sender)
P

tensor([[0.5000, 0.5000],
        [1.0000, 0.0000],
        [0.0000, 1.0000],
        [0.0000, 1.0000]])


tensor([[1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 0.]])

In [17]:
# Receiver update
# PoReceiver = np.dot(PRho, Confusion) # P_o(t_o|m)
# PSigmaInverse = makePDFPerRow(Priors * np.transpose(PSigma))
# ExpUR = np.array([
#     [np.sum([PSigmaInverse[m, ta] * ConfusionR[ti, tr] * Utility[ta, tr] for ta in xrange(NStates) for tr in
#                 xrange(NStates)])
#         for ti in xrange(NStates)] for m in xrange(NMessages)])
# Hearer = makePDFPerRow(PoReceiver * ExpUR)

O_receiver = Rho @ C
sigma_inverse = normalize_rows(prior * Sigma.T)
# U_receiver = torch.tensor([
#     [
#         torch.sum(
#             [
#                 sigma_inverse[w,a] * C[i, r] * U[a,r] 
#              for a in range(num_states) for r in range(num_states)
#             ]) 
#         for i in range(num_states)] 
#     for w in range(num_signals)
# ])

U_receiver = torch.einsum('wa,ia,ar->wi', sigma_inverse, C, U)

Q = normalize_rows(O_receiver * U_receiver)